#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=29376 sha256=8be3c52a8385b47da8418b0bc259b8fa97b8d07d482f11a4fbc560861f8d6cfa
  Stored in directory: /tmp/pip-ephem-wheel-cache-bak1opca/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [2]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [3]:
from boston311 import Boston311KerasNN, Boston311LogReg


2024-05-11 15:02:15.864065: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 15:02:17.395539: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 15:02:17.416714: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 15:02:31.149171: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Load extra features

In [4]:
today_datestring, tomorrow_datestring, thirty_days_ago_datestring = Boston311KerasNN().get_datestrings()

In [5]:
import glob
import os

def get_latest_model_files(model_dir="./daily_models/Boston311KerasNN/"):
    # Get all json and h5 files
    json_files = glob.glob(os.path.join(model_dir, "*.json"))
    h5_files = glob.glob(os.path.join(model_dir, "*.h5"))
    #also add .keras files to h5_files
    keras_files = glob.glob(os.path.join(model_dir, "*.keras"))
    h5_files.extend(keras_files)

    # Sort files by modification time
    json_files.sort(key=os.path.getmtime, reverse=True)
    h5_files.sort(key=os.path.getmtime, reverse=True)

    if json_files and h5_files:
        latest_json = json_files[0]
        latest_h5 = h5_files[0]
        return latest_json, latest_h5
    else:
        return None, None

# Use the function
json_file, model_file = get_latest_model_files()
print(json_file)

if json_file and model_file:
    KerasNN_model = Boston311KerasNN()
    KerasNN_model.load(json_file, model_file)
    KerasNN_model.predict_date_range['end'] = tomorrow_datestring


./daily_models/Boston311KerasNN/2024-01-01_15-39-39_Boston311KerasNN.json


In [6]:
data = Boston311LogReg(train_date_range={'start':'2024-01-01','end':'2024-12-31'}).load_data()
#save data to csv as all_311_cases.csv
data.to_csv('all_311_cases.csv', index=False)
#save data to a pkl file as all_311_cases.pkl
data.to_pickle('all_311_cases.pkl')

Checking files_dict
files_dict is None
trying to call get311URLs
trying to get csv URLs
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/dff4d804-5031-443a-8409-8344efd0e5c8/download/tmpepr4mhz1.csv for year 2024
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmp182oee02.csv for year 2023
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv for year 2022
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv for year 2021
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv for year 2020
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resourc

In [7]:
case_data_file = 'case_data.pkl'
case_data_csv = 'all_311_cases.csv'
data = KerasNN_model.pkl_load_data(case_data_csv, case_data_file)

In [8]:

#parse CLS embedding column as array
predictions, prediction_data = KerasNN_model.predict(data=data)

columns in data before drop: Index(['case_enquiry_id', 'open_dt', 'sla_target_dt', 'closed_dt', 'on_time',
       'case_status', 'closure_reason', 'case_title', 'subject', 'reason',
       'type', 'queue', 'department', 'submitted_photo', 'closed_photo',
       'location', 'fire_district', 'pwd_district', 'city_council_district',
       'police_district', 'neighborhood', 'neighborhood_services_district',
       'ward', 'precinct', 'location_street_name', 'location_zipcode',
       'latitude', 'longitude', 'geom_4326', 'source', 'survival_time',
       'event', 'ward_number', 'survival_time_hours'],
      dtype='object')
columns to drop: Index(['case_status', 'case_title', 'city_council_district', 'closed_dt',
       'closed_photo', 'closure_reason', 'fire_district', 'geom_4326',
       'latitude', 'location', 'location_street_name', 'location_zipcode',
       'longitude', 'neighborhood', 'neighborhood_services_district',
       'on_time', 'open_dt', 'police_district', 'precinct', 'pwd_

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:267: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, fake_df], ignore_index=True)
2024-05-11 15:03:08.967490: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 186693184 exceeds 10% of free system memory.


2989/2989 [==============================] - 15s 4ms/step


2024-05-11 15:03:40.595836: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23336648 exceeds 10% of free system memory.


In [9]:
# Define a function to flatten an array into a string.
def array_to_string(arr):
    return ' '.join(map(str, arr))

# Apply the function along axis 1 (rows).
string_predictions = np.apply_along_axis(array_to_string, axis=1, arr=predictions)

# Now string_predictions is a 1D NumPy array where each element is a string
# that contains all the elements from the corresponding row in the original 2D array.
print(string_predictions)

['0.09663813 0.022824474 0.010438997 0.0066735586 0.0058646053 0.003218923 0.0023069426 0.0014355038 0.0011967091 0.0011953643 0.0011134399 0.0009689744 0.00066344405 0.0010612286 0.0009876881 0.0013261486 0.0023285134 0.0025271338 0.0021400985 0.0023529928 0.0031748882 0.004121762 0.004243303 0.003988161 0.004038441 0.0036580535 0.0048188074 0.0036540893 0.0039568855 0.004027564 0.003670396 0.0039120843 0.0023643048 0.005267582 0.005564161 0.0037782816 0.0032201987 0.005868544 0.004114565 0.003088083 0.0044334484 0.0056719678 0.003632142 0.004363965 0.0057843635 0.0055135177 0.0035351939 0.0057612164 0.00495946 0.0042247493 0.00525867 0.0058860513 0.0050674244 0.0035718328 0.008186262 0.0074522696 0.004590026 0.0047570807 0.0076746726 0.0050252452 0.64685744'
 '0.991101 0.0064599155 0.00076955534 0.0002155851 9.491766e-05 9.7460725e-05 9.511811e-05 5.7471952e-05 3.0164152e-05 3.7016587e-05 4.6906836e-05 5.9604394e-05 4.3576492e-05 4.202112e-05 3.0660445e-05 2.462565e-05 3.8313465e-05 

In [10]:
case_enquiry_id = prediction_data['case_enquiry_id']

In [11]:
#combine case_enquiry_id and predictions into a dataframe
predictions_df = pd.DataFrame({'case_enquiry_id':case_enquiry_id, 'prediction':string_predictions})

In [12]:
import pandas as pd

#get model_name from json_file name and ml_model_date from json_file name first 8 characters which are YYYY-MM-DD and keep it to YYYY-MM-DD
model_name = json_file.split('/')[-1].split('.')[0]
ml_model_date = json_file.split('/')[-1].split('.')[0][:10]

#define an empt pandas dataframe ml_model_df
ml_model_df = pd.DataFrame(columns=['ml_model_name', 'ml_model_type', 'ml_model_date'])

#read contents of json_file into ml_model_json
with open(json_file, 'r') as f:
    ml_model_json = f.read()

ml_model_df = pd.concat([ml_model_df, pd.DataFrame([{'ml_model_name': model_name, 
                                    'ml_model_type': KerasNN_model.model_type,
                                    'ml_model_date': ml_model_date}])], ignore_index=True)

print(ml_model_df)

                          ml_model_name     ml_model_type ml_model_date
0  2024-01-01_15-39-39_Boston311KerasNN  Boston311KerasNN    2024-01-01


In [13]:
all_model_cases = prediction_data.drop(['geom_4326','survival_time_hours', 'survival_time', 'event'], axis=1).copy()

In [14]:

all_model_predictions = predictions_df

In [15]:
all_model_predictions['ml_model_name'] = model_name

In [16]:
# %%
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [17]:
all_model_predictions['prediction_date'] = today_datestring

In [18]:


# %%
all_model_cases.to_csv(my_datetime+'_311_cases.csv', index=False)


# %%

all_model_predictions.to_csv(my_datetime+'_311_predictions.csv', index=False)

# %%

ml_model_df.to_csv(my_datetime+'_311_ml_models.csv', index=False)

# %%
#create datetime _manifest.txt file with one filename per line
with open(my_datetime+'_manifest.txt', 'w') as f:
    f.write(my_datetime+'_311_cases.csv\n')
    f.write(my_datetime+'_311_ml_models.csv\n')
    f.write(my_datetime+'_311_predictions.csv\n')

# %%
#create an export folder

SCP_COMMAND = "scp -i /home/briarmoss/.ssh/id_rsa_new" 
EXPORT_FOLDER = "briarmoss@10.0.0.81:/home/briarmoss/Documents/BODC-DEI-site/database/seeders/"

#copy the csv files to the export folder
#! {SCP_COMMAND} {my_datetime}_311_cases.csv {EXPORT_FOLDER}
#! {SCP_COMMAND} {my_datetime}_311_predictions.csv {EXPORT_FOLDER}
#! {SCP_COMMAND} {my_datetime}_311_ml_models.csv {EXPORT_FOLDER}
#! {SCP_COMMAND} {my_datetime}_manifest.txt {EXPORT_FOLDER}



# %% [markdown]
# ** Copy the files to the production server **

# %%


# %%







In [19]:
import os

#my_datetime = '2024-05-08_11-09-36'

# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"{my_datetime}_311_cases.csv",
    f"{my_datetime}_311_ml_models.csv",
    f"{my_datetime}_311_predictions.csv",
    f"{my_datetime}_manifest.txt"
]

# Control where to copy
copy_to_prod = True
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


# %%
PORT_NUMBER

Executing: scp -P 65002 2024-05-11_15-03-54_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2024-05-11_15-03-54_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2024-05-11_15-03-54_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2024-05-11_15-03-54_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2024-05-11_15-03-54_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2024-05-11_15-03-54_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2024-05-11_15-03-54_manifest.txt u353344964@195.179.236.61:

65002

In [20]:

if copy_to_prod:
    ! ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

if copy_to_stage:
    ! ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'


   INFO  Seeding database.  


Manifest files:
/home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2024-05-11_15-03-54_manifest.txt

Total records to process: 191285

Processing /home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2024-05-11_15-03-54_311_cases.csv
100 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 95542.
Total records remaining: 191185.
Time for last 100 records: 0.09 seconds.
Estimated time remaining for this file: 1 minute, 26 seconds.
Estimated time for all files: 2 minutes, 52 seconds.
200 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 95442.
Total records remaining: 191085.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 5 seconds.
Estimated time for all files: 11 seconds.
300 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 95342.
Total records remaining: 190985.
Time for last 100 records: 0.01 seconds.
Estimated time r